In [10]:
import os
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras import utils as np_utils

In [11]:
train_data_folder='Dataset/train/'
test_data_folder='Dataset/test/'
train_files=os.listdir(train_data_folder)

In [12]:
complete_data=None
string_to_int={}
int_to_string={}
counter=0

def fill_data(filename):
    data=np.load(file=train_data_folder+filename)
    filename=filename.split('.')[0]
    temp_var=-1
    
    global counter
    global complete_data
    global string_to_int
    global int_to_string
    
    if filename in string_to_int:
        temp_var=string_to_int[filename]
    
    else:
        temp_var=counter
        counter+=1
        string_to_int[filename]=temp_var
        int_to_string[temp_var]=filename
    
    dimension=data.shape
    temp_y=np.ones(shape=(dimension[0],1),dtype=np.float64)*temp_var
    data=np.concatenate((data,temp_y),axis=1)
    
    if complete_data is None:
        complete_data=np.copy(data)
    
    else:
        complete_data=np.concatenate((complete_data,data),axis=0)
    

for filename in train_files:
    fill_data(filename)

print(complete_data.shape)
print(string_to_int)
print(int_to_string)    

(100000, 785)
{'keyboard': 3, 'harp': 14, 'banana': 2, 'foot': 19, 'penguin': 13, 'spider': 11, 'skyscraper': 0, 'snowman': 10, 'trombone': 16, 'hand': 17, 'chair': 6, 'hat': 4, 'flashlight': 9, 'violin': 7, 'nose': 1, 'laptop': 18, 'eyeglasses': 5, 'bulldozer': 12, 'pig': 15, 'parrot': 8}
{0: 'skyscraper', 1: 'nose', 2: 'banana', 3: 'keyboard', 4: 'hat', 5: 'eyeglasses', 6: 'chair', 7: 'violin', 8: 'parrot', 9: 'flashlight', 10: 'snowman', 11: 'spider', 12: 'bulldozer', 13: 'penguin', 14: 'harp', 15: 'pig', 16: 'trombone', 17: 'hand', 18: 'laptop', 19: 'foot'}


In [13]:
complete_data=np.random.permutation(complete_data)
print(complete_data.shape)

(100000, 785)


In [14]:
[instances,dimensions]=complete_data.shape

x_train=complete_data[:,0:dimensions-1].astype(np.float32)
test_x=np.load(file="Dataset/test/test.npy").astype(np.float32)
y_train2=complete_data[:,dimensions-1:].astype(np.int32)
y_train = np_utils.to_categorical(y_train2)
print(x_train.shape)
print(y_train.shape)
print(y_train2.shape)
print(test_x.shape)
categories=20

(100000, 784)
(100000, 20)
(100000, 1)
(100000, 784)


In [15]:
# param=1000
# x_train=x_train[0:param,:]
# y_train=y_train[0:param]
# y_train.shape
x_train/=255.0
test_x/=255.0
x_train=x_train.reshape((100000,28,28,1))
test_x=test_x.reshape((100000,28,28,1))

In [ ]:
model = Sequential()
model.add(Conv2D(64, (5, 5), padding='same', activation='relu', input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(categories, activation='softmax'))

In [ ]:
batch_size = 256
epochs = 100
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1,validation_split=0.05)
# model1.evaluate(test_data, test_labels_one_hot)

Train on 95000 samples, validate on 5000 samples
Epoch 1/100
95000/95000 [==============================] - 6s 60us/step - loss: 1.1546 - acc: 0.6752 - val_loss: 0.7177 - val_acc: 0.8070
Epoch 2/100
95000/95000 [==============================] - 5s 48us/step - loss: 0.6114 - acc: 0.8279 - val_loss: 0.5788 - val_acc: 0.8296
Epoch 3/100
95000/95000 [==============================] - 5s 47us/step - loss: 0.4945 - acc: 0.8588 - val_loss: 0.5325 - val_acc: 0.8468
Epoch 4/100
95000/95000 [==============================] - 5s 48us/step - loss: 0.4339 - acc: 0.8747 - val_loss: 0.4953 - val_acc: 0.8564
Epoch 5/100
95000/95000 [==============================] - 5s 48us/step - loss: 0.3925 - acc: 0.8863 - val_loss: 0.4687 - val_acc: 0.8612
Epoch 6/100
95000/95000 [==============================] - 5s 48us/step - loss: 0.3611 - acc: 0.8940 - val_loss: 0.4633 - val_acc: 0.8634
Epoch 7/100
95000/95000 [==============================] - 5s 47us/step - loss: 0.3264 - acc: 0.9039 - val_loss: 0.4559 - v

In [ ]:
def label_to_string(label):
    lis=[]
    instances=label.shape[0]
    for i in range(instances):
        lis.append(int_to_string[label[i]])
    return lis

In [ ]:
predictions=np.argmax(model.predict(test_x),axis=1)
print(predictions.shape)
predictions=label_to_string(predictions)
predictions=np.asarray(predictions)
print(predictions.shape)

In [ ]:
df=pd.DataFrame(data=predictions,columns=['CATEGORY'])
df.index.name='ID'
df.to_csv(path_or_buf="ignore/CNN_sub.csv",sep=',')